<h1>Deep Neural Network for MNIST Classification</h1>

We'll apply all the knowledge from the lectures in this section to write a deep neural network. The problem we've chosen is referred to as the "Hello World" of deep learning because for most students it is the first deep learning algorithm they see.

The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). He is one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as covolutional neural networks (CNNs). 

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes. 

Our goal would be to build a neural network with 2 hidden layers.

<h2>Import the relevant packages</h2>

In [1]:
import numpy as np
import tensorflow as tf

from datetime import datetime
import tensorflow_datasets as tfds

<h2>Data</h2>

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validations_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validations_samples = tf.cast(num_validations_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

print(f"number of validations samples: {num_validations_samples} ")
print(f"number of test samples: {num_test_samples} ")

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validations_samples)
train_data = shuffled_train_and_validation_data.skip(num_validations_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validations_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

number of validations samples: 6000 
number of test samples: 10000 


<h2>Model</h2>

<h2>Outline the model</h2>

In [3]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

<h2>Choose the optimizer and the loss function</h2>

In [4]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

<h2>Training</h2>

In [5]:
NUM_EPOCHS = 7

logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2, callbacks=[tensorboard_callback])

Epoch 1/7
540/540 - 3s - loss: 0.3293 - accuracy: 0.9043 - val_loss: 0.1565 - val_accuracy: 0.9580
Epoch 2/7
540/540 - 2s - loss: 0.1372 - accuracy: 0.9598 - val_loss: 0.1210 - val_accuracy: 0.9675
Epoch 3/7
540/540 - 2s - loss: 0.0947 - accuracy: 0.9719 - val_loss: 0.0829 - val_accuracy: 0.9767
Epoch 4/7
540/540 - 2s - loss: 0.0747 - accuracy: 0.9770 - val_loss: 0.0714 - val_accuracy: 0.9802
Epoch 5/7
540/540 - 2s - loss: 0.0576 - accuracy: 0.9822 - val_loss: 0.0665 - val_accuracy: 0.9817
Epoch 6/7
540/540 - 2s - loss: 0.0484 - accuracy: 0.9855 - val_loss: 0.0587 - val_accuracy: 0.9828
Epoch 7/7
540/540 - 2s - loss: 0.0401 - accuracy: 0.9878 - val_loss: 0.0466 - val_accuracy: 0.9867


<h2>Test the model</h2>

In [6]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 229ms/step - loss: 0.0778 - accuracy: 0.9766


In [7]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.08. Test accuracy: 97.66%
